# Gotcha's from Pandas to Dask

This notebook highlights some key differences when transfering code from Pandas to run in a Dask environment.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
# client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client = Client()
client

Client Scheduler: tcp://127.0.0.1:43778 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 67.44 GB


## Create 2 DataFrames: 
1. for Dask 
2. for Pandas  
Dask comes with builtin dataset samples.  
In order to create a Pandas dataframe all that is needed is to run compute()

In [2]:
import dask
import pandas as pd
print(f'Dask versoin: {dask.__version__}')
print(f'Pandas versoin: {pd.__version__}')

Dask versoin: 1.2.0
Pandas versoin: 0.24.2


In [3]:
ddf = dask.datasets.timeseries()
ddf

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


* Remember `Dask dataframes` are **lazy** thus in order to see the result we need to run compute() 
 (or head which runs under the hood compute()) )

In [4]:
ddf.head()

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1053,Edith,-0.082485,0.126788
2000-01-01 00:00:01,992,Zelda,-0.292002,0.814259
2000-01-01 00:00:02,991,Yvonne,-0.435019,-0.923308
2000-01-01 00:00:03,998,Xavier,-0.164077,0.195232
2000-01-01 00:00:04,989,Norbert,0.343440,-0.787839


In [6]:
pdf = ddf.compute()  # create a pandas dataframe
print(type(pdf))
pdf.head()

<class 'pandas.core.frame.DataFrame'>


,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1053,Edith,-0.082485,0.126788
2000-01-01 00:00:01,992,Zelda,-0.292002,0.814259
2000-01-01 00:00:02,991,Yvonne,-0.435019,-0.923308
2000-01-01 00:00:03,998,Xavier,-0.164077,0.195232
2000-01-01 00:00:04,989,Norbert,0.343440,-0.787839


Now that we have both `dataframes` we can start to compair the interactions with them

## 1. Conceptual shift - from Update to Insert/Delete
Dask does not update - thus no arguments such as "inplace= True" which exist in Pandas.  
For more detials see [issue#653 on github](https://github.com/dask/dask/issues/653)


### 1.1 Rename

In [ ]:
# Pandas
print(pdf.columns)
pdf.rename(columns={'id':'ID','name':'Name','x':'coor_x', 'y':'coor_y'},inplace=True)
pdf.columns

In [ ]:
# Dask
# Must update using the correct sequence of the columns
print(ddf.columns)
ddf.columns = ['ID','Name','coor_x','coor_y']
ddf.columns

## 1.2 Column munipilations  
There are several diffrences when manimuplating data. mose of which require to ran a couple of lines (instead of one-liners)

### Convert index into Time column

In [ ]:
# Pandas
pdf = pdf.assign(Time=pd.to_datetime(pdf.index).time)
pdf.head()

In [ ]:
# Dask
ddf = ddf.assign(Time=ddf.index)
ddf['Time'] = ddf['Time'].dt.time
ddf.head()

## 1.3 Drop NA on column

In [ ]:
# Pandas
pdf = pdf.assign(colna = None)
print(pdf.head())
pdf.dropna(axis=1, how='all', inplace=True)
print(pdf.head())

In [ ]:
# Dask
ddf = ddf.assign(colna = None)
print(ddf.head())
if ddf.colna.isnull().all().compute() == True:   # check if all values in column are Null - VERY slow
    ddf = ddf.drop(labels=['colna'],axis=1)
print(ddf.head())

##  1.4 Reset Index

In [ ]:
# Pandas
pdf.reset_index(drop=True, inplace=True)
pdf.head()

Dask is in a development mode
thus there bugs that are fixed all the time   
e.g. [reset_index fails when index is named ](https://github.com/dask/dask/pull/4509)

In [ ]:
# This currently fails without reseting the dataframe......
ddf = dask.datasets.timeseries()

In [ ]:
# Dask
ddf.index.name = None   # workaround
ddf = ddf.reset_index()
ddf['Time'] = ddf['timestamp'].dt.time
ddf = ddf.drop(labels=['timestamp'], axis=1 )
ddf.columns = ['ID','Name','coor_x','coor_y','Time']
ddf.head()

# 2. Reads/Save files

When working with pandas and dask preferable try and work with parquet.  
Even so when working with Dask - the files can be read with multiple workers 

### Save files

In [ ]:
# Pandas
!mkdir data
pdf.to_csv('data/pdf_single_file.csv')
!ls data

In [ ]:
# Dask
# 1. notice the '*' to allow for multiple file renaming. all kwrgs are applicable
# 2. notice that the path to the directory may change based on the location of the running notebook
ddf.to_csv('data/pd2dd/ddf*.csv', index = False)
!ls data/pd2dd/
# to fild number of partitions use dask.dataframe.npartitions

### Read files

based on an [answer from stack overflow](https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe)

In [ ]:
# Pandas 
path = r'data/'                     # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)

In [ ]:
# Dask
ddf = dask.dataframe.read_csv('data/pd2dd/ddf*.csv')
ddf.head()

Most `kwarg` are available for reading and writing 
e.g. 
ddf = dask.dataframe.read_csv('data/pd2dd/ddf*.csv', compression='gzip', header none)
However `nrows` is not available


## 3. Group By
In addition to the notebook example that is in the repository - 
This is another example how to try to eliminate the use of `groupby.apply`  
In this example we are grouping by coloumns into unique list.

In [ ]:
# prepare pandas dataframe
pdf = pdf.assign(Time=pd.to_datetime(pdf.index).time)
pdf['seconds'] = pdf.Time.astype(str).str[-2:]
pdf.head()

In [ ]:
# pandas preperations
def set_list_att(x: dask.dataframe.Series):
        return list(set([item for item in x.values]))

In [ ]:
%%timeit
# pandas option 1 using apply
pdf_gb = pdf.groupby(pdf.Name)
gp_col = ['ID', 'seconds']
list_ser_gb = [pdf_gb[att_col_gr].apply(set_list_att) for att_col_gr in gp_col]
df_edge_att = pdf_gb.size().to_frame(name="Weight")
for ser in list_ser_gb:
        df_edge_att = df_edge_att.join(ser.to_frame(), how='left')        
df_edge_att.head()

In [ ]:
%%timeit
# pandas option 2 using lambda
pdf_gb = pdf.groupby(pdf.Name)
gp_col = ['ID', 'seconds']
list_ser_gb = [pdf_gb[att_col_gr].apply(lambda x: list(set(x.to_list()))) for att_col_gr in gp_col]
df_edge_att = pdf_gb.size().to_frame(name="Weight")
for ser in list_ser_gb:
        df_edge_att = df_edge_att.join(ser.to_frame(), how='left')        
df_edge_att.head()

In any case sometimes using Pandas is more efficiante (assuming that you can load all the data into the RAM).  
In this case Pandas is faster

In [ ]:
# prepare dask dataframe
ddf['seconds'] = ddf.Time.astype(str).str[-2:]
ddf = client.persist(ddf)
ddf.head()

In [ ]:
%%timeit
# Dask option1 using apply
# notice the meta argument in the apply function
df_gb = ddf.groupby(ddf.Name)
gp_col = ['ID', 'seconds']
list_ser_gb = [df_gb[att_col_gr].apply(set_list_att
                                      ,meta=pd.Series(dtype='object', name=f'{att_col_gr}_att')) 
               for att_col_gr in gp_col]
df_edge_att = df_gb.size().to_frame(name="Weight")
for ser in list_ser_gb:
        df_edge_att = df_edge_att.join(ser.to_frame(), how='left')        
df_edge_att.head()

Using [dask custom aggregation](https://docs.dask.org/en/latest/dataframe-api.html?highlight=dropna#dask.dataframe.groupby.Aggregation) is consideribly better

In [ ]:
# Dask
# some preperations
import itertools
custom_agg = dask.dataframe.Aggregation(
    'custom_agg', 
    lambda s: s.apply(set), 
    lambda s: s.apply(lambda chunks: list(set(itertools.chain.from_iterable(chunks)))),
)

In [ ]:
%%timeit
# Dask option1 using apply
df_gb = ddf.groupby(ddf.Name)
gp_col = ['ID', 'seconds']
list_ser_gb = [df_gb[att_col_gr].agg(custom_agg) for att_col_gr in gp_col]
df_edge_att = df_gb.size().to_frame(name="Weight")
for ser in list_ser_gb:
        df_edge_att = df_edge_att.join(ser.to_frame(), how='left')        
df_edge_att.head()

 ## 4. Consider using Persist
Since Dask is lazy - it may ran the **entire** graph (again) even if it already ran part of it in order to generate a result 
in a previous cell.  
```python
ddf = client.persist(ddf)
```
This is different from Pandas which once a variable was created it will keep all data in memory.  
Additional information can be read in this [stackoverflow issue](https://stackoverflow.com/questions/45941528/how-to-efficiently-send-a-large-numpy-array-to-the-cluster-with-dask-array/45941529#45941529) or see an exampel in [this post](http://matthewrocklin.com/blog/work/2017/01/12/dask-dataframes)   
This concept should also  be used when running a code within a script (rather then a jupyter notebook) which incoperates a loop logic within the code.

## 5. Debugging
Debugging my be more challenging since
1. when using a client - mutliprocessing is complecated
2. sometime introducing a faulty command into a graph (such as in a jupyter notebook) requirues to cache-out the graph and start the process from the begining